In [1]:
import numpy as np
import pandas as pd

np.random.seed(0)
x = np.random.randint(10, size=(5000,1))
y = np.random.randint(2, size=(5000,1))

data = pd.DataFrame(np.concatenate([y,x],axis=1), columns=['y','x'])
data.head()

x = np.random.randint(10, size=(1000000,1))
y = np.random.randint(2, size=(1000000,1))

data_100w = pd.DataFrame(np.concatenate([y,x],axis=1), columns=['y','x'])
data_100w.head()

,y,x
0,1,3
1,0,6
2,1,9
3,1,2
4,1,6


In [9]:
# 讲义中的 v2 版本作为 base line
def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
#     print(value_dict)
#     print(count_dict)
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [85]:
%timeit target_mean_v2(data, 'y', 'x')

341 ms ± 16.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# 1. Python 原生优化

## v3 提取变量优化
y, x = data.loc[i] 慢

In [89]:
def target_mean_v3(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        y, x = data.loc[i]
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[x] = y
            count_dict[x] = 1
        else:
            value_dict[x] += y
            count_dict[x] += 1
#     print(value_dict)
#     print(count_dict)
    for i in range(data.shape[0]):
        y, x = data.loc[i]
        result[i] = (value_dict[x] - y) / (count_dict[x] - 1)
    return result

## v3a 提取变量优化
y, x = data.loc[i] 慢，y, x = data.loc[i, y_name],data.loc[i, x_name] 快，why？

In [90]:
def target_mean_v3a(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        y, x = data.loc[i, y_name],data.loc[i, x_name]
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[x] = y
            count_dict[x] = 1
        else:
            value_dict[x] += y
            count_dict[x] += 1
#     print(value_dict)
#     print(count_dict)
    for i in range(data.shape[0]):
        y, x = data.loc[i, y_name],data.loc[i, x_name]
        result[i] = (value_dict[x] - y) / (count_dict[x] - 1)
    return result

In [91]:
%timeit target_mean_v3(data, 'y', 'x')
%timeit target_mean_v3a(data, 'y', 'x')

1.19 s ± 37.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
254 ms ± 3.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## v3b 尝试 defaultdict （时快时慢）

In [177]:
from collections import defaultdict

def target_mean_v3b(data, y_name, x_name):
    length = data.shape[0]
    result = np.zeros(length)
    value_dict = defaultdict(int)
    count_dict = defaultdict(int)
    for i in range(length):
        y, x = data.loc[i, y_name],data.loc[i, x_name]
        value_dict[x] += y
        count_dict[x] += 1

    for i in range(length):
        y, x = data.loc[i, y_name],data.loc[i, x_name]
        result[i] = (value_dict[x] - y) / (count_dict[x] - 1)
        
    return result

## v3c 尝试 setdefault

In [179]:
def target_mean_v3c(data, y_name, x_name):
    length = data.shape[0]
    result = np.zeros(length)
    value_dict = {}
    count_dict = {}
    for i in range(length):
        y, x = data.loc[i, y_name],data.loc[i, x_name]
        value_dict[x] = value_dict.setdefault(x, 0) + y
        count_dict[x] = count_dict.setdefault(x, 0) + 1

    for i in range(length):
        y, x = data.loc[i, y_name],data.loc[i, x_name]
        result[i] = (value_dict[x] - y) / (count_dict[x] - 1)
        
    return result

In [181]:
%timeit target_mean_v3a(data, 'y', 'x')
%timeit target_mean_v3b(data, 'y', 'x')
%timeit target_mean_v3c(data, 'y', 'x')

# defaultdict 简化了代码，但速度变慢了, setdefault 简化了代码，且速度变快了

256 ms ± 7.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
222 ms ± 8.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
212 ms ± 5.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## v3d 尝试 groupby（简化了代码，但变慢了）

In [45]:
# 计算总体的 sum 和count
groupby_result = data.groupby(['x'], as_index=False).agg(['sum', 'count'])
groupby_result

y      
   sum count
x           
0  259   506
1  247   502
2  251   478
3  233   517
4  241   490
5  275   510
6  240   502
7  260   491
8  271   506
9  276   498

In [59]:
sum, count = groupby_result.loc[9]
sum, count

(276, 498)

In [188]:
from collections import defaultdict

def target_mean_v3d(data, y_name, x_name):
    length = data.shape[0]
    result = np.zeros(length)
    groupby_result = data.groupby(['x'], as_index=False).agg(['sum', 'count'])

    for i in range(length):
        y, x = data.loc[i, y_name],data.loc[i, x_name]
        sum_total, count_total = groupby_result.loc[x]
        result[i] = (sum_total - y) / (count_total - 1)
    return result

In [182]:
%timeit target_mean_v3d(data, 'y', 'x')

889 ms ± 32.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## v4 DataFrame中的loc操作较慢,使用数组替换

In [6]:
def target_mean_v4(data, y_name, x_name):
    length = data.shape[0]
    result = np.zeros(length)
    xs, ys = data[x_name].tolist(), data[y_name].tolist()
    value_dict = {}
    count_dict = {}
    for i in range(length):
        y, x = ys[i], xs[i]
        value_dict[x] = value_dict.setdefault(x, 0) + y
        count_dict[x] = count_dict.setdefault(x, 0) + 1

    for i in range(length):
        y, x = ys[i], xs[i]
        result[i] = (value_dict[x] - y) / (count_dict[x] - 1)
    
#     print(value_dict, count_dict)
    return result

In [202]:
result = target_mean_v4(data, 'y', 'x')
print(result)
print(result.shape)

[0.53831041 0.51089109 0.45155039 ... 0.47704591 0.52410901 0.55331992]
(5000,)


In [127]:
import numpy as np

def target_mean_v4a(data, y_name, x_name):
    length = data.shape[0]
    result = np.zeros(length)
    xs, ys = data[x_name].values, data[y_name].values
    value_dict = {}
    count_dict = {}
    for i in range(length):
        y, x = ys[i], xs[i]
        value_dict[x] = value_dict.setdefault(x, 0) + y
        count_dict[x] = count_dict.setdefault(x, 0) + 1

    for i in range(length):
        y, x = ys[i], xs[i]
        result[i] = (value_dict[x] - y) / (count_dict[x] - 1)
        
    return result

In [128]:
%timeit target_mean_v4(data, 'y', 'x')
%timeit target_mean_v4a(data, 'y', 'x')
# tolist 比 values 版本更快，可能是 list 比 ndarray 索引数据快

3.61 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
9.86 ms ± 153 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# 2. 使用 Cython 优化

In [2]:
%load_ext Cython

## cython 指定变量类型

In [ ]:
%%cython -a

import numpy as np
cimport numpy as cnp
import array

cdef target_mean_v5_cython(data, str y_name, str x_name):
    cdef int length = data.shape[0]
    cdef cnp.ndarray[double] result = np.zeros(length)
    xs = data[x_name].tolist()
    ys = data[y_name].tolist()
    value_dict = dict()
    count_dict = dict()

    for i in range(length):
        y, x = ys[i], xs[i]
        value_dict[x] = value_dict.setdefault(x, 0) + y
        count_dict[x] = count_dict.setdefault(x, 0) + 1

    for i in range(length):
        y, x = ys[i], xs[i]
        result[i] = (value_dict[x] - y) / (count_dict[x] - 1)
        
    return result

In [ ]:
%%cython -a

import numpy as np
cimport numpy as cnp

cdef target_mean_v5a_cython(data, str y_name, str x_name):
    cdef int length = data.shape[0]
    cdef cnp.ndarray[double] result = np.zeros(length)
    cdef cnp.ndarray[long] xs = data[x_name].values
    cdef cnp.ndarray[long] ys = data[y_name].values
    value_dict = dict()
    count_dict = dict()

    for i in range(length):
        y, x = ys[i], xs[i]
        value_dict[x] = value_dict.setdefault(x, 0) + y
        count_dict[x] = count_dict.setdefault(x, 0) + 1

    for i in range(length):
        y, x = ys[i], xs[i]
        result[i] = (value_dict[x] - y) / (count_dict[x] - 1)
        
    return result

In [276]:
%timeit target_mean_v5_cython(data, 'y', 'x')
%timeit target_mean_v5a_cython(data, 'y', 'x')
# Cython 下 cnp.ndarry 比 list 索引数据快

1.48 ms ± 33.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.24 ms ± 6.48 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 去除类型检查、包装检查

In [ ]:
%%cython -a

import numpy as np
cimport numpy as cnp
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cdef target_mean_v5b_cython(data, str y_name, str x_name):
    cdef int length = data.shape[0]
    cdef cnp.ndarray[double] result = np.zeros(length)
    cdef cnp.ndarray[long] xs = data[x_name].values
    cdef cnp.ndarray[long] ys = data[y_name].values
    value_dict = dict()
    count_dict = dict()
    
    for i from 0 <= i < length by 1:
        y, x = ys[i], xs[i]
        value_dict[x] = value_dict.setdefault(x, 0) + y
        count_dict[x] = count_dict.setdefault(x, 0) + 1

    for i from 0 <= i < length by 1:
        y, x = ys[i], xs[i]
        result[i] = (value_dict[x] - y) / (count_dict[x] - 1)
        
    return result

In [155]:
%timeit target_mean_v5a_cython(data, 'y', 'x')
%timeit target_mean_v5b_cython(data, 'y', 'x')

1.31 ms ± 37.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.3 ms ± 23.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 使用 memoryview

In [ ]:
%%cython -a

import numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cdef target_mean_v5c_cython(data, str y_name, str x_name):
    cdef int length = data.shape[0]
    cdef double[:] result = np.zeros(length)
    cdef long[:] xs = data[x_name].values
    cdef long[:] ys = data[y_name].values
    cdef long[:] value = np.zeros(10).astype(long)
    cdef long[:] count = np.zeros(10).astype(long)
    
    for i from 0 <= i < length by 1:
        y, x = ys[i], xs[i]
        value[x] += y
        count[x] += 1

    for i from 0 <= i < length by 1:
        y, x = ys[i], xs[i]
        result[i] = (value[x] - y) / (count[x] - 1)
        
    return result

In [158]:
%timeit target_mean_v5c_cython(data, 'y', 'x')

747 µs ± 25.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 使用 prange openmp 并行

In [ ]:
%%cython -a

import numpy as np
cimport cython
from cython.parallel import prange

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v5d_cython(data, str y_name, str x_name):
    cdef int length = data.shape[0]
    cdef double[:] result = np.zeros(length)
    cdef long[:] xs = data[x_name].values
    cdef long[:] ys = data[y_name].values
    cdef long[:] value = np.zeros(10).astype(long)
    cdef long[:] count = np.zeros(10).astype(long)
    
    cdef int i = 0
    for i in prange(length, nogil=True):
        value[xs[i]] += ys[i]
        count[xs[i]] += 1

    for i in prange(length, nogil=True):
        result[i] = (value[xs[i]] - ys[i]) / (count[xs[i]] - 1)
        
    return result

In [113]:
%timeit target_mean_v5d_cython(data, 'y', 'x')
%timeit target_mean_v5d_cython(data_100w, 'y', 'x')

55.3 µs ± 1.75 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
7.85 ms ± 125 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# 3. 使用 Numba 优化

In [4]:
import numpy as np
import numba 
from numba import njit
        
@njit
def _target_mean_v6(length, xs, ys, y_name, x_name):
    result = np.empty(length)
    value = np.zeros(10)
    count = np.zeros(10)
        
    for i in range(length):
        y, x = ys[i], xs[i]
        value[x] += y
        count[x] += 1

    for i in range(length):
        y, x = ys[i], xs[i]
        result[i] = (value[x] - y) / (count[x] - 1)
        
    return result


def target_mean_v6_numba(data, y_name, x_name):
    length = data.shape[0]
    xs, ys = data[x_name].values, data[y_name].values
    return _target_mean_v6(length, xs, ys, y_name, x_name)

In [185]:
%timeit target_mean_v6_numba(data, 'y', 'x')
%timeit target_mean_v6_numba(data_100w, 'y', 'x')

53.7 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
6.56 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
# 检查优化后的版本的结果是否正确
r1 = target_mean_v4(data, 'y', 'x')
r2 = target_mean_v5d_cython(data, 'y', 'x')
r3 = target_mean_v6_numba(data, 'y', 'x')

print(np.linalg.norm(r2 - r1))
print(np.linalg.norm(r3 - r1))

0.0
0.0


In [10]:
# 运行环境：本地 MacBook Pro

# 讲义的 v2 版本
%timeit target_mean_v2(data, 'y', 'x')

# python 原生最快的版本
%timeit target_mean_v4(data, 'y', 'x')
# python 原生最快的版本 100w
%timeit target_mean_v4(data_100w, 'y', 'x')

# cython 最快的版本 5k
%timeit target_mean_v5d_cython(data, 'y', 'x')
# cython 最快的版本 100w
%timeit target_mean_v5d_cython(data_100w, 'y', 'x')

# numba 版本 5k
%timeit target_mean_v6_numba(data, 'y', 'x')
# cython 版本 100w
%timeit target_mean_v6_numba(data_100w, 'y', 'x')

325 ms ± 7.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.55 ms ± 54.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
740 ms ± 5.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
55.3 µs ± 2.06 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
7.55 ms ± 53.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
52.5 µs ± 2.56 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
7.2 ms ± 59.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# 4. 总结

python 原生优化：

* 使用简单的数据结构，数组取数据比 DataFrame 快
* 使用简单的算法，自己实现的比自带的 groupby 快, setdefault 比 defaultdict 快

Cython 优化:

* 指定变量类型
* 使用 memoryview
* 使用 prange 并行

Numba 优化:

* 针对 numpy 的优化，numba 最小的改动就能达到跟Cython 一样好的优化效果